In [1]:
import random
import shutil
from http import HTTPStatus
from pathlib import Path
from typing import List

import cv2 as cv
import requests

from fishsense_gmm_laser_detector.config import settings

In [2]:
n_frames_to_select = 5
rgb = [255, 0, 0]
v_limit = 200

In [3]:
output_path = Path('./images')
if output_path.exists():
    shutil.rmtree(output_path)
output_path.mkdir(parents=True, exist_ok=True)

In [4]:
spot_path = Path('./spots')
if spot_path.exists():
    shutil.rmtree(spot_path)
spot_path.mkdir(parents=True, exist_ok=True)

In [5]:
headers = {
    'api_key': settings.e4efs.api_key
}

In [6]:
with requests.Session() as session:
    session.headers.update(headers)
    dive_list = session.get('https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/metadata/dives').json()['dives']
    frames: List[str] = []
    for dive in dive_list:
        dive_frames = session.get(f'https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/metadata/dive/{dive}').json()['frames']
        frames.extend(dive_frames)

In [7]:
frames_to_select = []
random.seed(0)
with requests.Session() as session:
    session.headers.update(headers)
    for idx in range(n_frames_to_select):
        img = None
        while img is None:
            checksum = random.choice(frames)
            if checksum in frames_to_select:
                continue
            laser_label_req = session.get(f'https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/laser/{checksum}')
            if laser_label_req.status_code != HTTPStatus.OK:
                continue
            laser_label = laser_label_req.json()
            img_req = session.get(f'https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/preprocess_jpeg/{checksum}')
            if img_req.status_code != HTTPStatus.OK:
                continue
            jpeg_path = output_path / f'{checksum}.jpg'
            with open(jpeg_path, 'wb') as handle:
                handle.write(img_req.content)
            img_data = cv.imread(jpeg_path)
            rgb_data = cv.cvtColor(img_data, cv.COLOR_BGR2RGB)
            hsv_data = cv.cvtColor(img_data, cv.COLOR_BGR2HSV)
            pixel_value = rgb_data[laser_label['x'], laser_label['y']]
            # is R bigger than others?
            if not (pixel_value[0] > pixel_value[1] and pixel_value[0] > pixel_value[2]):
                # Not a red laser
                continue
            # This is a red laser
            if hsv_data[laser_label['x'], laser_label['y'], 2] > v_limit:
                # too bright
                continue
            img = checksum
        frames_to_select.append((img, laser_label))
frames_to_select

[('eaf23b033d7739306134ca7df532bdf2',
  {'task_id': 43023, 'x': 1972, 'y': 1395}),
 ('6b735932f8f5571c6e005ccaec41f9a1',
  {'task_id': 39693, 'x': 1943, 'y': 1251}),
 ('dd514a27994a301d1ebe26f0b0d1c2b6',
  {'task_id': 44878, 'x': 2025, 'y': 1293}),
 ('97ef263638239355c9448d4c53b73d5f',
  {'task_id': 41418, 'x': 1908, 'y': 1235}),
 ('5e4ad064821097b7a1bfff8c24dc56c3',
  {'task_id': 40271, 'x': 2054, 'y': 1393})]

In [8]:
boxes = [
    {
        'checksum': 'eaf23b033d7739306134ca7df532bdf2',
        'x0': 1969,
        'y0': 1390,
        'x1': 1969+9,
        'y1': 1390+11
    },
    {
        'checksum': '6b735932f8f5571c6e005ccaec41f9a1',
        'x0': 1936,
        'y0': 1239,
        'x1': 1936+17,
        'y1': 1239+22
    },
    {
        'checksum': 'dd514a27994a301d1ebe26f0b0d1c2b6',
        'x0': 2021,
        'y0': 1286,
        'x1': 2021+8,
        'y1': 1286+11
    },
    {
        'checksum': '97ef263638239355c9448d4c53b73d5f',
        'x0': 1902,
        'y0': 1227,
        'x1': 1902+12,
        'y1': 1227+19
    },
    {
        'checksum': '5e4ad064821097b7a1bfff8c24dc56c3',
        'x0': 2051,
        'y0': 1387,
        'x1': 2051+8,
        'y1': 1387+13
    }
]

In [9]:
for box in boxes:
    bgr_img = cv.imread(output_path / f'{box['checksum']}.jpg')
    pixels = bgr_img[box['y0']:box['y1'], box['x0']:box['x1']]
    cv.imwrite(spot_path / f'{box['checksum']}.jpg', pixels)